# コード生成のために Bedrock モデルを呼び出す

> *このノートブックは SageMaker Studio の **`Data Science 3.0`** カーネルをご利用ください*

## 概要

このノートでは、LLM を使ってテキスト・プロンプトに基づいたコードを生成する方法を紹介します。

Boto3 API を使用して、Bedrock の Claude v2 を使用します。

この例で使用されているプロンプトは、プロンプト以外のテキストの例を提供していないため、ゼロショットプロンプトと呼ばれています。

**注意:** *このノートブックはAWS環境内外でも実行可能です*

#### コンテキスト
Amazon Bedrock のコード生成機能を実証するために、Amazon Bedrock API と通信するための Boto3 クライアントの使用を検討します。利用可能なさまざまな構成と、単純な入力がどのように望ましい出力につながるかを実演します。

#### パターン
Amazon Bedrock API に、タスク、命令、入力からなる入力を提供するだけで、追加の例を提供することなく、フードの下のモデルが出力を生成します。ここでの目的は、強力な LLM がいかに簡単に手元のタスクを理解し、説得力のある出力を生成するかを示すことです。

![](./images/bedrock-code-gen.png)

#### ユースケース
Amazon Bedrock におけるモデルの生成機能を示すために、コード生成のユースケースを取り上げましょう。

#### ペルソナ

あなたは AnyCompany のデータアナリスト、Moe です。この会社では、過去 1 年間のさまざまな製品の販売実績を把握したいと考えています。あなたは、sales.csv というデータセットを提供されています。このデータセットには以下の列が含まれています。:

- Date (YYYY-MM-DD) format
- Product_ID (各製品に固有な値)
- Price (各製品が実際に売れた値段)

#### 実装
このユースケースを実現するために、このノートブックでは、与えられたプロンプトに対してコードを生成する方法を示します。Amazon Bedrock API と Boto3 クライアントを使用して、Anthropic Claude v2 を使用します。

## セットアップ

このノートブックの残りの部分を実行する前に、以下のセルを実行して（必要なライブラリーがインストールされていることを確認し）Bedrockに接続する必要があります。

セットアップの仕組みや ⚠️ **変更が必要かについては**,  [Bedrock boto3 setup](../00_Intro/bedrock_boto3_setup.ipynb) ノートブックを参照してください。

In [ ]:
# 事前にリポジトリルートから `download-dependencies.sh` を実行済みであることを確認してください!
%pip install --no-build-isolation --force-reinstall \
    ../dependencies/awscli-*-py3-none-any.whl \
    ../dependencies/boto3-*-py3-none-any.whl \
    ../dependencies/botocore-*-py3-none-any.whl

%pip install --quiet langchain==0.0.249

In [ ]:
# このノートブックで生成されたコードを実行するには
%pip install matplotlib

In [ ]:
import json
import os
import sys

import boto3

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww


# ---- ⚠️ 必要に応じて AWS 設定に関する以下のコードのコメントを解除、編集してください ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."
# os.environ["BEDROCK_ENDPOINT_URL"] = "<YOUR_ENDPOINT_URL>"  # E.g. "https://..."


boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    endpoint_url=os.environ.get("BEDROCK_ENDPOINT_URL", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
)

## コード生成

上で説明したユースケースに続いて、ユースケースのための Python プログラムを生成するために、Amazon Bedrock サービスへの入力を準備しましょう。

#### Lab セットアップ - このラボで使用するサンプル sales.csv データを作成します。



In [ ]:
# sales.csv ファイルを作成する
import csv

data = [
    ["date", "product_id", "price", "units_sold"],
    ["2023-01-01", "P001", 50, 20],
    ["2023-01-02", "P002", 60, 15],
    ["2023-01-03", "P001", 50, 18],
    ["2023-01-04", "P003", 70, 30],
    ["2023-01-05", "P001", 50, 25],
    ["2023-01-06", "P002", 60, 22],
    ["2023-01-07", "P003", 70, 24],
    ["2023-01-08", "P001", 50, 28],
    ["2023-01-09", "P002", 60, 17],
    ["2023-01-10", "P003", 70, 29],
    ["2023-02-11", "P001", 50, 23],
    ["2023-02-12", "P002", 60, 19],
    ["2023-02-13", "P001", 50, 21],
    ["2023-02-14", "P003", 70, 31],
    ["2023-03-15", "P001", 50, 26],
    ["2023-03-16", "P002", 60, 20],
    ["2023-03-17", "P003", 70, 33],
    ["2023-04-18", "P001", 50, 27],
    ["2023-04-19", "P002", 60, 18],
    ["2023-04-20", "P003", 70, 32],
    ["2023-04-21", "P001", 50, 22],
    ["2023-04-22", "P002", 60, 16],
    ["2023-04-23", "P003", 70, 34],
    ["2023-05-24", "P001", 50, 24],
    ["2023-05-25", "P002", 60, 21]
]

# sales.csv に書き込む
with open('sales.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(data)

print("sales.csv has been created!")

#### Amazon Bedrock が生成した Python プログラムによる売上分析

In [ ]:
# プロンプトを作成する
# 売上分析

prompt_data = """

Human: You have a CSV, sales.csv, with columns:
- date (YYYY-MM-DD)
- product_id
- price
- units_sold

Create a python program to analyze the sales data from a CSV file. The program should be able to read the data, and determine below:

- Total revenue for the year
- The product with the highest revenue
- The date with the highest revenue
- Visualize monthly sales using a bar chart

Ensure the code is syntactically correct, bug-free, optimized, not span multiple lines unnessarily, and prefer to use standard libraries. Return only python code without any surrounding text, explanation or context.

Assistant:
"""

まずは Anthropic Claude V2 モデルから使ってみましょう

In [43]:
# Claude - Body 文
body = json.dumps({
                    "prompt": prompt_data,
                    "max_tokens_to_sample":4096,
                    "temperature":0.5,
                    "top_k":250,
                    "top_p":0.5,
                    "stop_sequences": ["\n\nHuman:"]
                  }) 

#### Anthropic Claude v2 モデルを呼び出す

In [ ]:
modelId = 'anthropic.claude-v2' # モデルプロバイダーごとに変更
accept = 'application/json'
contentType = 'application/json'

response = boto3_bedrock.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
response_body = json.loads(response.get('body').read())

print_ww(response_body.get('completion'))

#### (オプション) 検証のためにBedrockが生成したコードを実行しましょう。生成されたコードをテキストエディタにコピーしてください。以下のセルのコードを置き換えてください。

In [ ]:
# サンプル Python コード（前のステップで Amazon Bedrock で生成されたもの）

import csv
from collections import defaultdict
import matplotlib.pyplot as plt

revenue = 0
monthly_revenue = defaultdict(int)
product_revenue = defaultdict(int)
max_revenue = 0
max_revenue_date = ''
max_revenue_product = ''

with open('sales.csv') as f:
    reader = csv.reader(f)
    next(reader)
    for row in reader:
        date = row[0]
        product = row[1]
        price = float(row[2])
        units = int(row[3])

        revenue += price * units
        product_revenue[product] += price * units
        monthly_revenue[date[:7]] += price * units

        if revenue > max_revenue:
            max_revenue = revenue
            max_revenue_date = date
            max_revenue_product = product

months = list(monthly_revenue.keys())
values = list(monthly_revenue.values())

plt.bar(months, values)
plt.xlabel('Month')
plt.ylabel('Revenue')
plt.title('Monthly Revenue')
plt.show()

print('Total Revenue:', revenue)
print('Product with max revenue:', max_revenue_product)
print('Date with max revenue:', max_revenue_date)

## 結論
これで `boto3`SDK 経由で Bedrock API を 使うことが出来ました。この API を使用し、指定された販売データの分析および可視化の Python ブログラムを作成しました。

### 持ち帰って頂きたいこと
- このノートブックを調整し、Amazon Titan や AI21 Labs Jurassic モデルなど、Amazon Bedrock を通じて利用できるさまざまなモデルに差し替え、試してみてください。


## ありがとうございます